# Introduction

Set Up

In [79]:
# Standard imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from collections import Counter
from scipy import stats
from scipy.stats import norm

In [80]:
# show all dataframe columns
pd.set_option('display.max_columns', None)
# set matplotlib global settings eg. figsize
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [81]:
#import data
original_df = pd.read_csv('../data/rm_system/my_data.csv')

In [82]:
#quick look into the data
original_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [83]:
#checking dimensions of data
original_df.shape

(119390, 32)

There are 119,390 rows and 32 columns

**Creating Column Description Table for Reference**
| **Column**                     | **Description**                                                                                                                                                                                                                                                                                                                                                                                                                   |
|--------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| hotel                          |                                                                                                                                                                                                H1 = Resort Hotel or H2 = City Hotel                                                                                                                                                                                               |
| is_canceled                    |                                                                                                                                                                                    Value indicating if the booking was canceled (1) or not (0)                                                                                                                                                                                    |
| lead_time                      |                                                                                                                                                              Number of days that elapsed between the entering date of the   booking into the PMS and the arrival date                                                                                                                                                             |
| arrival_date_year              |                                                                                                                                                                                                        Year of arrival date                                                                                                                                                                                                       |
| arrival_date_month             |                                                                                                                                                                                                          Month   of arrival date                                                                                                                                                                                                  |
| arrival_date_week_number       |                                                                                                                                                                                                Week number of year for arrival date                                                                                                                                                                                               |
| arrival_date_day_of_month      |                                                                                                                                                                                                        Day of arrival date                                                                                                                                                                                                        |
| stays_in_weekend_nights        |                                                                                                                                                                  Number of weekend nights (Saturday or Sunday) the guest stayed   or booked to stay at the hotel                                                                                                                                                                  |
| stays_in_week_nights           |                                                                                                                                                                     Number of week nights (Monday to Friday) the guest stayed or   booked to stay at the hotel                                                                                                                                                                    |
| adults                         |                                                                                                                                                                                                          Number of adults                                                                                                                                                                                                         |
| children                       |                                                                                                                                                                                                         Number of children                                                                                                                                                                                                        |
| babies                         |                                                                                                                                                                                                          Number of babies                                                                                                                                                                                                         |
| meal                           |                                                                                  Type of meal booked. Categories are presented in standard   hospitality meal packages: Undefined/SC – no meal package; BB – Bed &   Breakfast; HB – Half board (breakfast and one other meal – usually dinner);   FB – Full board (breakfast, lunch and dinner)                                                                                  |
| country                        |                                                                                                                                                                           Country of origin. Categories are represented in the ISO   3155–3:2013 format                                                                                                                                                                           |
| market_segment                 |                                                                                                                                                          Market segment designation. In categories, the term “TA” means   “Travel Agents” and “TO” means “Tour Operators”                                                                                                                                                         |
| distribution_channel           |                                                                                                                                                                   Booking   distribution channel. The term “TA” means “Travel Agents” and “TO” means   “Tour Operators”                                                                                                                                                           |
| is_repeated_guest              |                                                                                                                                                                          Value indicating if the booking name was from a repeated guest   (1) or not (0)                                                                                                                                                                          |
| previous_cancellations         |                                                                                                                                                                   Number of previous bookings that were cancelled by the   customer prior to the current booking                                                                                                                                                                  |
| previous_bookings_not_canceled |                                                                                                                                                                      Number of previous bookings not cancelled by the customer   prior to the current booking                                                                                                                                                                     |
| reserved_room_type             |                                                                                                                                                                   Code of room type reserved. Code is presented instead of   designation for anonymity reasons.                                                                                                                                                                   |
| assigned_room_type             |                                                                              Code for the type of room assigned to the booking. Sometimes   the assigned room type differs from the reserved room type due to hotel   operation reasons (e.g. overbooking) or by customer request. Code is   presented instead of designation for anonymity reasons.                                                                              |
| booking_changes                |                                                                                                                                        Number of changes/amendments made to the booking from the   moment the booking was entered on the PMS until the moment of check-in or   cancellation                                                                                                                                       |
| deposit_type                   |                                                            Indication on if the customer made a deposit to guarantee the   booking. This variable can assume three categories: No Deposit – no deposit   was made; Non Refund – a deposit was made in the value of the total stay   cost; Refundable – a deposit was made with a value under the total cost of   stay.                                                            |
| agent                          |                                                                                                                                                                                       ID of the   travel agency that made the booking                                                                                                                                                                                             |
| company                        |                                                                                                                                       ID of the company/entity that made the booking or responsible   for paying the booking. ID is presented instead of designation for anonymity   reasons                                                                                                                                      |
| days_in_waiting_list           |                                                                                                                                                                    Number of days the booking was in the waiting list before it   was confirmed to the customer                                                                                                                                                                   |
| customer_type                  | Type of booking, assuming one of four categories: Contract -   when the booking has an allotment or other type of contract associated to it;   Group – when the booking is associated to a group; Transient – when the   booking is not part of a group or contract, and is not associated to other   transient booking; Transient-party – when the booking is transient, but is   associated to at least other transient booking |
| adr                            |                                                                                                                                                       Average Daily Rate as defined by dividing the sum of all   lodging transactions by the total number of staying nights                                                                                                                                                       |
| required_car_parking_spaces    |                                                                                                                                                                                       Number of car parking spaces required by the customer                                                                                                                                                                                       |
| total_of_special_requests      |                                                                                                                                                                          Number of special requests made by the customer (e.g. twin bed   or high floor)                                                                                                                                                                          |
| reservation_status             |                                                                                     Reservation last status, assuming one of three categories:   Canceled – booking was canceled by the customer; Check-Out – customer has   checked in but already departed; No-Show – customer did not check-in and did   inform the hotel of the reason why                                                                                    |
| reservation_status_date        |                                                                                                            Date at which the last status was set. This variable can be   used in conjunction with the ReservationStatus to understand when was the   booking canceled or when did the customer checked-out of the hotel                                                                                                           |

In [84]:
#Review datatypes and null values
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

**Initial Look at Columns**
|           **Column**           | **Non Null Count** | **Null Values** | **Type** |     **Preprocessing Step**    | **Potential Drop?** |
|:------------------------------:|:------------------:|:---------------:|:--------:|:-----------------------------:|:-------------------:|
| hotel                          |       119390       |        0        | object   |                               |                     |
| is_canceled                    |       119390       |        0        | int64    |                               |                     |
| lead_time                      |       119390       |        0        | int64    |                               |                     |
| arrival_date_year              |       119390       |        0        | int64    |                               |                     |
| arrival_date_month             |       119390       |        0        | object   | Convert to INT64              |                     |
| arrival_date_week_number       |       119390       |        0        | int64    |                               |                     |
| arrival_date_day_of_month      |       119390       |        0        | int64    |                               |                     |
| stays_in_weekend_nights        |       119390       |        0        | int64    |                               |                     |
| stays_in_week_nights           |       119390       |        0        | int64    |                               |                     |
| adults                         |       119390       |        0        | int64    |                               |                     |
| children                       |       119386       |        4        | float64  | Convert and impute nulls to 0 |                     |
| babies                         |       119390       |        0        | int64    |                               |                     |
| meal                           |       119390       |        0        | object   | Dummy Variable                |          Y          |
| country                        |       118902       |       488       | object   | Impute                        |                     |
| market_segment                 |       119390       |        0        | object   | Create Dummy Variables        |                     |
| distribution_channel           |       119390       |        0        | object   | Create Dummy Variables        |                     |
| is_repeated_guest              |       119390       |        0        | int64    | Binary                        |                     |
| previous_cancellations         |       119390       |        0        | int64    |                               |                     |
| previous_bookings_not_canceled |       119390       |        0        | int64    |                               |                     |
| reserved_room_type             |       119390       |        0        | object   | Create Dummy Variables        |                     |
| assigned_room_type             |       119390       |        0        | object   | Create Dummy Variables        |                     |
| booking_changes                |       119390       |        0        | int64    | Create Dummy Variables        |                     |
| deposit_type                   |       119390       |        0        | object   | Create Dummy Variables        |                     |
| agent                          |       103050       |      16340      | float64  | Create Dummy Variables        |                     |
| company                        |        6797        |      112593     | float64  | Create Dummy Variables        |                     |
| days_in_waiting_list           |       119390       |        0        | int64    |                               |          Y          |
| customer_type                  |       119390       |        0        | object   | Create Dummy Variables        |                     |
| adr                            |       119390       |        0        | float64  |                               |                     |
| required_car_parking_spaces    |       119390       |        0        | int64    |                               |          Y          |
| total_of_special_requests      |       119390       |        0        | int64    |                               |          Y          |
| reservation_status             |       119390       |        0        | object   | Binary/Dummy?                 |                     |
| reservation_status_date        |       119390       |        0        | object   | Binary/Dummy?                 |                     |

## Identify what columns can be quickly addressed
- `children` float -> int
- `arrival_date_month` object -> datetime month
- `country` impute nulls since dataset only has 2 hotels

In [85]:
#Creating copy of original DF before initial cleaning
init_clean_df = original_df.copy()

In [86]:
# checking how many nulls in children column
init_clean_df['children'].isna().sum()

4

In [87]:
# fill children null values to 0
init_clean_df['children'] = init_clean_df['children'].fillna(0)

In [88]:
#sanity check
init_clean_df['children'].isna().sum()

0

In [89]:
# Convert children column to int
init_clean_df['children']= init_clean_df['children'].astype(int)

In [90]:
#sanity check
init_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119390 non-null  int64  
 11  babies                          119390 non-null  int64  
 12  meal            

In [91]:
#Convert arrival_date_month to integers
#Create a dictionary to map month names to integers
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12}

#Use mapping to convert arrival_date_month to integer
init_clean_df['arrival_date_month'] = init_clean_df['arrival_date_month'].map(month_mapping)

In [92]:
#Sanity check
init_clean_df['arrival_date_month'].head()

0    7
1    7
2    7
3    7
4    7
Name: arrival_date_month, dtype: int64

In [93]:
#impute country nulls based on hotel column
init_clean_df['country'].value_counts()

country
PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
       ...  
DJI        1
BWA        1
HND        1
VGB        1
NAM        1
Name: count, Length: 177, dtype: int64

The value_counts above shows that this is not the country of the hotel, rather it is the country of origin of the guest. Of the columns that can be quickly addressed
## Quickly addressed columns checkpoint
- (COMPLETE) `children` float -> int 
- (COMPLETE)`arrival_date_month` object -> integer
- `country` impute nulls since dataset only has 2 hotels (Incorrect assumption of country variable. Next step is either dummy variable or delete the column. created a column description table above for reference)

In [94]:
init_clean_df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,6.552483,27.165173,15.798241,0.927599,2.500302,1.856403,0.103886,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,3.090619,13.605138,8.780829,0.998613,1.908286,0.579261,0.398555,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,4.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,7.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,9.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,12.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


## Identifying Suspicious Values
**Suspicious Values**
- Max `adults` is 55 - possible data entry error?
- Max `children` is 10 - possible data entry error?
- Max `babies` is 10 - possible data entry error
- Max `stays_in_weekend_nights` is 19. With only 2 night per weekend, that means this guest stayed for 9.5 weeks, equating to almost a 70night stay
- Max `previous_cancellations` is 26. Possible the guest booked and cancelled a lot/potentially a corporate traveller needing to change reservations frequently? Plausible for city hotel, not as likely for resort hotel..
- Min `adr` is -6.38. ADR cannot be negative. For each reservation sold, they are provided a rate.  Comp rooms are $0 and no situation would a guest be paid to stay in a room...
- `company` and `agent` should not be a float. Convert to either INT or categorical as dummy variables depending on how many unique values appear.

Looking into Adults column. Dataset source keeps the hotel locations anoymous for privacy reasons.  Only identifier is that these two properties are in Portugal.

In [95]:
#identifying the unique values counts for adults
init_clean_df['adults'].value_counts()

adults
2     89680
1     23027
3      6202
0       403
4        62
26        5
27        2
20        2
5         2
40        1
50        1
55        1
6         1
10        1
Name: count, dtype: int64

In [96]:
#Identifying what hotel these large length of stays are in 
#np.where(init_clean_df['adults'] > 7, init_clean_df['hotel'])

init_clean_df[init_clean_df['adults'] > 7]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
1539,Resort Hotel,1,304,2015,9,36,3,0,3,40,0,0,BB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1587,Resort Hotel,1,333,2015,9,36,5,2,5,26,0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1643,Resort Hotel,1,336,2015,9,37,7,1,2,50,0,0,BB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-18
1752,Resort Hotel,1,340,2015,9,37,12,2,5,26,0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1884,Resort Hotel,1,347,2015,9,38,19,2,5,26,0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1917,Resort Hotel,1,349,2015,9,39,21,1,3,27,0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
1962,Resort Hotel,1,352,2015,9,39,24,1,3,27,0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
2003,Resort Hotel,1,354,2015,9,39,26,2,5,26,0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
2164,Resort Hotel,1,361,2015,10,40,3,2,5,26,0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,96.0,NaN,0,Group,0.0,0,0,Canceled,2015-01-02
2173,Resort Hotel,1,338,2015,10,41,4,2,0,55,0,0,HB,PRT,Direct,Direct,0,0,0,A,A,0,No Deposit,NaN,NaN,0,Group,0.0,0,0,Canceled,2015-01-02


The above results when looking at only hotels where the number of adults exceed 7, indicates that these are group blocks (mostly cancelled), but that they are not assigned to a single room.  Group blocks can be placed and can exceed the capacity of a single room.  FYI the largest room in the world can only accommodate up to 10 guests. <br>
<br>
This observation may also be applied to the suspicious values for `babies` and `children`.

In [ ]:
#checking babies
init_clean_df[init_clean_df['babies'] > 5]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
46619,City Hotel,0,37,2016,1,3,12,0,2,2,0,10,BB,PRT,Online TA,TA/TO,0,0,0,D,D,1,No Deposit,9.0,NaN,0,Transient,84.45,0,1,Check-Out,2016-01-14


In [98]:
#Imputing the 2 values to be 0 due to likelihood of being a data entry error
init_clean_df[init_clean_df['babies'] > 5] = 0

In [104]:
#sanity check that anomalies were removed 
init_clean_df['babies'].value_counts()

babies
0    118475
1       900
2        15
Name: count, dtype: int64

In [100]:
#checking children
init_clean_df[init_clean_df['children'] > 5]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
328,Resort Hotel,1,55,2015,7,29,12,4,10,2,10,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,2,No Deposit,8.0,NaN,0,Contract,133.16,0,1,No-Show,2015-07-12


In [102]:
#imputing value to 0
init_clean_df[init_clean_df['children'] > 5] = 0

In [103]:
#sanity check
init_clean_df['children'].value_counts()

children
0    110801
1      4861
2      3652
3        76
Name: count, dtype: int64

**Suspicious Values Update**
- (Complete) Max `adults` is 55 - valid due to group blocks. however leads me to pivot to adjusting my target to strictly transient demand.
- (Complete) Max `children` is 10 - imputed with 0
- (Complete) Max `babies` is 10 - imputed with 1 or 0
- (Keep As Is) Max `stays_in_weekend_nights` is 19. With only 2 night per weekend, that means this guest stayed for 9.5 weeks, equating to almost a 70night stay
- Max `previous_cancellations` is 26. Possible the guest booked and cancelled a lot/potentially a corporate traveller needing to change reservations frequently? Plausible for city hotel, not as likely for resort hotel..
- Min `adr` is -6.38. ADR cannot be negative. For each reservation sold, they are provided a rate.  Comp rooms are $0 and no situation would a guest be paid to stay in a room...
- `company` and `agent` should not be a float. Convert to either INT or categorical as dummy variables depending on how many unique values appear.

In [108]:
#imputing any negative ADR to 0
init_clean_df[init_clean_df['adr'] < 0] = 0

In [ ]:
#sanity check, min is now 0 and not negative
init_clean_df['adr'].describe()

count    119390.000000
mean        101.828557
std          50.538147
min           0.000000
25%          69.242500
50%          94.540000
75%         126.000000
max        5400.000000
Name: adr, dtype: float64

In [ ]:
#checking value counts of stays in weekend nights - looks like there are some longer LOS's in this dataset
init_clean_df['stays_in_weekend_nights'].value_counts()

stays_in_weekend_nights
0     52001
2     33307
1     30626
4      1853
3      1259
6       153
5        79
8        60
7        19
9        11
10        7
12        5
13        3
16        3
14        2
18        1
19        1
Name: count, dtype: int64

In [ ]:
#checking value counts of previous cancellations - looks like it is normal in this dataset
init_clean_df['previous_cancellations'].value_counts()

previous_cancellations
0     112906
1       6051
2        116
3         65
24        48
11        35
4         31
26        26
25        25
6         22
19        19
5         19
14        14
13        12
21         1
Name: count, dtype: int64

**Suspicious Values Update**
- (Complete) Max `adults` is 55 - valid due to group blocks. however leads me to pivot to adjusting my target to strictly transient demand.
- (Complete) Max `children` is 10 - imputed with 0
- (Complete) Max `babies` is 10 - imputed with 1 or 0
- (Keep As Is) Max `stays_in_weekend_nights` is 19. Value counts show long stays are not impossible. Many 10+ weekend_nights
- (Keep As Is) Max `previous_cancellations` is 26. Value counts show that this is normal in this dataset.

- (Complete) Min `adr` is -6.38. Imputed negative adr to 0.<br>
<br>
- (Incomplete)`company` and `agent` should not be a float. Convert to either INT or categorical as dummy variables depending on how many unique values appear.

The next steps are to identify what categories require dummy variables:

In [54]:
#create variable for only categoric columns
df_cat = init_clean_df.select_dtypes(include='object').copy()

#create variable for only numeric columns
df_num = init_clean_df.select_dtypes(include=['int','float']).copy()

In [ ]:
original_df['meal'].value_counts()

In [ ]:
df_cat.nunique()

In [ ]:
for col in df_cat:
    print(col, df_cat[col].value_counts())


In [ ]:
original_df.nunique()



***

# Previous Topic 
## Action Plan
1. Review Columns
    - Look into date column
    - Look into categorical columns
    - Look into numeric columns
    - Drop redundant columns
2. Identify any null values 
    - Decide to delete/impute null values
3. Check for errors in data

## Initial Look at Values

**Remove Reduntant columns**
1. Drop redundant/uncessary columns (9 columns)<br>
<br> 

**Decide on Unsure Variables**

- `country` - If keeping, no change needed
- `province` - Most likely drop
- `reviews.rating` - If keeping, convert to ordinal since ratings only go from 1-5
- `reviews.title` - Maybe merge this into the `reviews.text` so I dont need to impute?


**When columns have been decided on and nulls have been addressed**
1. Identify `reviews.text` null values. Delete Rows? How do you delete a row?
2. Check for errors in the columns

In [ ]:
#Saving copy of original DF and dropping the 2 null columns
hotel_review_df = original_df.drop(columns=['reviews.doRecommend','reviews.id']).copy()

In [ ]:
#checking reviews.dorecommend and reviews.id are removed
hotel_review_df.info()

## Review Columns

***
## Looking into the Date Columns
- `reviews.dateAdded`= date the review was added to the database
- `reviews.date`= date the review was posted online <--this is the more important value

In [ ]:
#reviews.dateAdded and reviews.date
hotel_review_df[['reviews.dateAdded', 'reviews.date']]

In [ ]:
hotel_review_df['reviews.dateAdded'].describe()

In [ ]:
#convert reviews.dateadded to datetime
hotel_review_df['reviews.dateAdded'] = pd.to_datetime(hotel_review_df['reviews.dateAdded']).dt.date

#convert reviews.date to datetime
hotel_review_df['reviews.date'] = pd.to_datetime(hotel_review_df['reviews.date']).dt.date


In [ ]:
hotel_review_df.head()

In [ ]:
#Comparing the two date columns, they are not the same. 
hotel_review_df[['reviews.dateAdded', 'reviews.date']].head()

After looking up the definitions of these columns, `reviews.date` seems like more valuable than `reviews.dateAdded`. Possible actions:
- Create dummy variables for the null volumns for `reviews.date` where Null is 0 and not Null is 1
- Impute using the `reviews.dateAdded`
    - Before doing this, look at trends to see of nonnull reviews, how close were they to the reviews.date
- Drop null value rows in `reviews.date`

***
## Looking at the Categoric Columns


***
## Looking at the Numeric Columns